<div class="alert alert-block alert-info">
    
The idea is to create classes which are then used freely to customize the experiment and adapt complexity based on the research question.

So, we can test participants on several values of ISI, DEVIATION frequencies and magnitudes and TONES, but this is not manipulated on class-level, but on experiment parameter level. To create all stimuli used in a single experiment, we call the classes with all the values that we want to test. I could think of making a class of function for making this too (using **list comprehensions for all possible values for two ranges of values & adding conditions** (rather than removing from all_combos). Also, see **how to define variables in one line with conditionals**!

### TONE

Use the tone module from expyriment.

#### Jitter

N (μ = 0, σ = E[|δn |]/ 2π) trun-
cated at δn ∈ [−320, 320] ms, with E[|δn |] ∈ {5, 10, 15, 20, 30, 40, 50, 60, 75, 90, 105, 120, 135, 150, 175, 200} ms

### TIMING DEVIATIONS: DEVIL VIOLATIONS

#### Creating a deviation one by one.

In [ ]:
class Devil():
    
    def __init__(self, magnitude, direction, location): 
        self.magnitude = magnitude # How big of a devil is it in msec?
        self.direction = direction # Prolonging or Shortening or Haciendo Nada with the ISI: late, early, on-time.
        self.location  = location  # To which sequential ISI in the sequence does the devil apply?
        # self.frequency = frequency  # How many devils do we have per sequence?
        
    def __str__(self):
        return f"Timing of tone no. {self.location + 1} is {self.direction} by {self.magnitude} msec."

In [ ]:
d = Devil(20, "early", 2) # 2nd ISI is manipulated, meaning that the 3rd tone is either late or early.
print(d.magnitude)
print(d.direction)
print(d.location)
print(d)

#### Creating a list of deviations.

In [ ]:
# Sampling the delay by which the onset of one tone is pertrubed in a sequence as written in the ERC: "Delays will be sampled from a 
# distribution spanning δ ∈ [−300, 300] ms in increments of 10 ms, plus four extra delays with values δ = ±5 ms, ±15 ms, resulting in
# a total of 64 delays."

class Devil:
    def __init__(self, start=-300, end=300, step=10):
        self.start = start
        self.end   = end
        self.step  = step

        self.devils = []
        self._generate_range()

    def _generate_range(self):
        for i in range(self.start, self.end, self.step):
            self.devils.append(i)
        self.devils.sort()
        
    def add_deviation(self, magnitude):
        for m in magnitude:
            self.devils.append(m)
        self.devils.sort()

    def get_random_dev(self, n):
        return random.sample(self.devils, n)

In [ ]:
sample = Devil(start=-300, end=300, step=10)

# Add custom deviations
sample.add_deviation([5, -5, 15, -15])

# Print list of all magnitudes
print(sample.devils)
print(sample.get_random_dev(4))


### SEQUENCE = STIMULUS

In [ ]:
class Sequence():
    
    def __init__(self, isi, tone, no_tones):
        self.isi  = isi
        self.tone = tone
        self.no_tones = no_tones
        
    def __str__(self):
        return f"A sequence with {self.isi}-msec-long inter-stimulus-intervals and {self.no_tones} tones in total."
    
    def create_noDev(self):
        seq = []
        
        # Sequence starts with the onset of the first tone and ends with the offset of the last tone.
        # Alternates tone / ISI / tone / ISI ...
        for _ in range(self.no_tones):
            seq.append(self.tone)
            
            if i < self.no_tones - 1:
                seq.append(self.isi)
        
        return seq
    
    def create_withDev(self, devil):
        seq = []
        deviation = devil.get_random_dev(1)
        for _ in range(self.no_tones):
            seq.append(self.tone)
            c += 1
            
            if c < self.no_tones: # Start with 1st tone's onset and end with the last's tone offset.
                
                if devil.location == c:
                    if devil.direction == "early":
                        isi_early = self.isi - deviation
                        seq.append(isi_early)

                    elif devil.direction == "late":
                        isi_late = self.isi + deviation
                        seq.append(isi_late)
                else:
                    seq.append(self.isi)
        
        return seq

In [ ]:
import random
b = Sequence(200, random.sample(range(50), 1), 10)
print(b.tone)
print(b.isi)
print(b.no_tones)
print(b)

c = b.create_withDev(d)
print("\n",c)

### TRIAL

In [ ]:
# IsequenceI: Inter Sequence Interval (time between two sequences, i.e., trials).

class Trial(): ############ COPY THE CLASS OF EXPYRIMENT IN THE REAL CODE ----------------------------------- !!
    
    def __init__(self, seq_number, InterSeqInterval, stimulus):
        self.seq_number = seq_number
        self.InterSeqInterval = InterSeqInterval
        self.stimulus = stimulus
        
    def __str__(self):
        return f"The trial has {self.seq_number} stimuli, each separated by {self.InterSeqInterval}. Stimulus is: {self.stimulus}"
    
    def create(self):
        
        stimuli_per_trial = []
        c = 0
        for _ in range(self.seq_number):
            stimuli_per_trial.append(self.stimulus)
            c += 1
            if c < self.seq_number:
                stimuli_per_trial.append(self.InterSeqInterval)
        
        return stimuli_per_trial

In [ ]:
t = Trial(3, 3000, c)
print(t.seq_number)
print(t.InterSeqInterval)
print(t)

t_all = t.create()
print(t_all)

### BLOCK

In [ ]:
# ITI: Inter Trial Interval (time between two trials).

class Block(): ############ COPY THE CLASS OF EXPYRIMENT IN THE REAL CODE ----------------------------------- !!
    
    def __init__(self, trial_number, iti, trial):
        self.trial_number = trial_number
        self.iti = iti
        self.trial = trial
        
    def __str__(self):
        return f"The block has {self.trial_number} trials, each separated by {self.iti} msec."
    
    def create(self):
        stimuli_per_block = []
        c = 0
        for _ in range(self.trial_number):
            stimuli_per_block.append(self.trial)
            c += 1
            
            if c < self.trial_number:
                stimuli_per_block.append(self.iti)
        
        return stimuli_per_block

In [ ]:
bl = Block(3, 3000, c)
print(bl.trial_number)
print(bl.iti)
print(bl)

bl_all = bl.create()
print(bl_all)

### TEST

In [ ]:
seq_200 = Sequence(200, random.sample(range(50), 1), 6) # isi, tone, no_tones
seq_250 = Sequence(250, random.sample(range(50), 1), 6) # isi, tone, no_tones

deviance_10 = Devil("early", 10, 4) # type, size, loc (isi no.)
seq_200 = seq_200.create_withDev(deviance_10)
seq_250 = seq_250.create_noDev()

trial_20_200 = Trial(20, 400, seq_200) # seq_number, InterSeqInterval, stimulus)
trial_20_250 = Trial(20, 500, seq_250) # seq_number, InterSeqInterval, stimulus)
trial_20_200 = trial_20_200.create()
trial_20_250 = trial_20_250.create()

block_200 = Block(2, 2000, trial_20_200) # trial_number, iti, trial
block_250 = Block(2, 2000, trial_20_250) # trial_number, iti, trial
block_200 = block_200.create()
block_250 = block_250.create()

print(block_200)
print(block_250)